In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [8]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [9]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [10]:
def R5_test(single):
    logger.info(f"Test-R5: Reward given out to block signers sums up to the total block reward")
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    while last_block - block > 32:
        block = getBlockNumber()
    if block == last_block:
        logger.info(f"current at the last block, wait until the 6th block in the new epoch")
        while block < last_block+6:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")

    acc_rewards_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated

    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    acc_rewards_curr = dict()
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    validator_rewards = 0
    total_reward = 0
    signers = 0
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            signers += 1
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            key_metrics = i['metrics']['by-bls-key']
            validator_reward = 0
            for by_key in key_metrics:
                validator_addr = by_key['key']['earning-account']
                by_key_reward = block_reward * float(by_key['key']['overall-percent']) / 0.32
                validator_reward += by_key_reward
            acc_rewards_curr[address] = reward_accumulated
            reward = reward_accumulated
            if address in acc_rewards_prev:
                reward -= acc_rewards_prev[address]
            total_reward += reward
            validator_rewards += validator_reward
    if format(total_reward, '.20e') != format(validator_rewards, '.20e'):        
        logger.warning(f"Test-R5: Fail")
        logger.warning(f"block: {block}, validator block reward: {validator_rewards:.20e}, total reward: {total_reward:.20e}, signers: {signers}\n")
        flag = False
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [11]:
R5_test(True)

INFO:economic-test:Test-R5: Reward given out to block signers sums up to the total block reward
INFO:economic-test:current and last block numbers: 128796, 128819
INFO:economic-test:current block 128796, will begin collecting infos...


current and last block numbers 128796 128819


INFO:economic-test:new block 128797 reached, will begin testing...



(False, None)